In [70]:
import os
path = os.getcwd() + '\\'
igai_initialize = path + 'IGAI_Initialize.ipynb'

%run $igai_initialize

06-02-2021 16:09:26 - Libraries and functions loaded correctly


In [71]:
minute = datetime.datetime.now().minute

if minute < 15:
    account = 'rebolbear'
elif minute < 30:
    account = 'reboludeces'
elif minute < 45:
    account = 'rebolbear'
else:
    account = 'reboludeces'

In [1]:
printf('Account: @' + account)
f, orig_stdout = saveLogs(account, 'FollowUnfollow')

driver = startDriver(account)
Login(account)

NameError: name 'printf' is not defined

# Pipeline

In [73]:
#Parameters

if account == 'reboludeces':
    max_requests_unfollow = 30 #30
    popularity_ratio_unfollow = 0
    max_requests_follow = 50   #50
    popularity_ratio_follow = 10
    shortcode = 'CKq11qHAqy7'
    min_following_days = 10

elif account == 'rebolbear':
    max_requests_unfollow = 40
    popularity_ratio_unfollow = 1.2
    max_requests_follow = 20
    popularity_ratio_follow = 10
    shortcode = 'CG8uZq8jX8n' # future 'CG93KEhDPqE'
    min_following_days = 10

    
#UNFOLLOW --------------------------------------------------------------------------------------------------------

printf('UNFOLLOW PROCESS OF @' + account.upper() + '----------------------------------------------------------')
relations = pd.read_csv(relations_file)
relations.loc[:,'time'] = relations['inserted_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'))
#users2unfollow = relations[relations.followed_by_username == account].sort_values('inserted_time')['username']
relations = relations[relations.time <  datetime.datetime.now() - datetime.timedelta(days=min_following_days)]
fwr = list(relations.loc[(relations.username == account) & (relations.status == "following"),"followed_by_username"])
fwd = list(relations.loc[(relations.followed_by_username == account) & (relations.status == "following"),"username"])
notfollowers = [item for item in fwd if item not in fwr]
followers = [item for item in fwd if item in fwr]
df_notfollowers = relations[(relations.username.isin(notfollowers)) & (relations.followed_by_username == account)].sort_values("time", ascending = True)
df_followers = relations[(relations.username.isin(followers)) & (relations.followed_by_username == account)].sort_values("time", ascending = True)
users2unfollow = pd.concat([df_notfollowers["username"], df_followers["username"]]) 

printf( '@' + account + ' has a list of ' + str(len(users2unfollow)) + ' users to unfollow')

try:
    followUnfollowTheseUsers(users2unfollow, process='unfollow', max_requests=max_requests_unfollow, popularity_ratio=popularity_ratio_unfollow)
except Exception as e:
    printf(e)
    printf('There was an error in the unfollow run')
    updateRelationsTable(relationstrk_file, relations_file)
    
    
#FOLLOW --------------------------------------------------------------------------------------------------------

print('\n')
printf('FOLLOW PROCESS OF @' + account.upper() + '---------------------------------------------------------------')
likestrk =  pd.read_csv(likestrk_file) 
users2follow = likestrk.loc[likestrk.shortcode == shortcode,"username"]

try:
    followUnfollowTheseUsers(users2follow, process='follow',  max_requests=max_requests_follow, popularity_ratio =popularity_ratio_follow)
except Exception as e:
    printf(e)
    printf('There was an error in the follow run')
    updateRelationsTable(relationstrk_file, relations_file)

print(' ')
printf('FINISHED FOLLOW PROCESS OF @' + account.upper() + '------------------------------------------------------')

06-02-2021 16:09:39 - UNFOLLOW PROCESS OF @REBOLBEAR----------------------------------------------------------
06-02-2021 16:09:40 - @rebolbear has a list of 2155 users to unfollow


C:\Users\DavidRebollo\Anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


06-02-2021 16:09:40 - The number of requests performed in the last 6 hours is 40
06-02-2021 16:09:41 - Max_requests exceeded: 40
06-02-2021 16:09:41 - Loaded file C:\Users\DavidRebollo\0.InstagramScrapping\dev\relationstrk with 138300 records.
06-02-2021 16:09:41 - Loaded file C:\Users\DavidRebollo\0.InstagramScrapping\dev\relations with 70585 records.
06-02-2021 16:09:42 - Table relations updated, now it has 70585 records.
06-02-2021 16:09:42 - Saved csv file: C:\Users\DavidRebollo\0.InstagramScrapping\dev\relations


06-02-2021 16:09:42 - FOLLOW PROCESS OF @REBOLBEAR---------------------------------------------------------------
06-02-2021 16:09:42 - The number of requests performed in the last 6 hours is 20
06-02-2021 16:09:43 - Max_requests exceeded: 20
06-02-2021 16:09:43 - Loaded file C:\Users\DavidRebollo\0.InstagramScrapping\dev\relationstrk with 138300 records.
06-02-2021 16:09:43 - Loaded file C:\Users\DavidRebollo\0.InstagramScrapping\dev\relations with 70585 records.
06-02-

In [74]:
#Truncate timestamp to date
#relations.loc[:,'date'] = relations['inserted_time'].apply(lambda x: datetime.datetime.strftime(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'),'%Y-%m-%d'))

#Frequency table
#pd.value_counts(user_relations.date).to_frame().reset_index()


In [75]:
driver.quit()
sys.stdout = orig_stdout
f.close()

AttributeError: 'int' object has no attribute 'close'